In [1]:
from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

from bs4 import BeautifulSoup
import re
import json

def get_article(articles):
    result = {}
    current_statue = "(non-statute)"
    for i in re.split(r"(.*)", articles.strip()):
        if len(i) == 0 or i == "\n":
            continue
        if re.search(r"^\(.*\)$", i):
            current_statue = i.strip()
            if current_statue not in result:
                result.update({current_statue: []})
        else:
            if current_statue not in result:
                result.update({current_statue: []})
            result[current_statue].append(i)
    return result

def build_test(filename):
    result = {}
    with open(filename, 'r') as f:
        data = f.read()

    data = BeautifulSoup(data, "xml").find_all('pair')
    for i in data:
        id = i.get('id')
        result.update({id: {}})
        result[id].update({"label": i.get('label')})
        articles = i.find('t1').text.strip()
        # articles = get_article(articles)
        result[id].update({"result": articles})
        result[id].update({"content": i.find('t2').text.strip()})
    return result

def write_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

import xml.etree.ElementTree as Et
import glob

def format_first_line(text):
    lines = text.split("\n")
    results = []
    for line in lines:
        if line[0] == "":
            continue
        if line[0] == "(" and line[-1] == ")":
            continue
        results.append(line)
    return "\n".join(results)

def load_samples(filexml):
    # try:
    tree = Et.parse(filexml)
    root = tree.getroot()
    samples = []
    for i in range(0, len(root)):
        sample = {'result': []}
        for j, e in enumerate(root[i]):
            if e.tag == "t1":
                sample['result'] = format_first_line(e.text.strip())
            elif e.tag == "t2":
                question = e.text.strip()
                sample['content'] = question if len(question) > 0 else None
        sample.update(
            {'index': root[i].attrib['id'], 'label': root[i].attrib.get('label', "N")})
        # filter the noise samples
        if sample['content'] is not None:
            samples.append(sample)
        else:
            print("[Important warning] samples {} is ignored".format(sample))
    return samples

def load_test_data_samples(path_folder_base, test_id):
    data = []
    test = load_samples(f"{path_folder_base}/riteval_{test_id}.xml")
    for file_path in glob.glob(f"{path_folder_base}/riteval_{test_id}.xml"):
        data = data + load_samples(file_path)
    return data


def load_all_data_samples(path_folder_base):
    data = []
    for file_path in glob.glob("{}/*.xml".format(path_folder_base)):
        data = data + load_samples(file_path)
    return data

def check_false_labels(pred, false_labels):
	for label in false_labels:
		if label in pred:
			return True
	return False

from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def readfile(filename):
    f = open(filename)
    data = json.load(f)
    return data

In [2]:
# finetuned = "fewshot_query_peft_results_prompt20_batchauto_epochs40"
output_data = "../data/COLIEE2024statute_data-English"

In [3]:
# zeroShot = "Document: {{premise}}\nQuestion: {{hypothesis}}? True or False "
# import jsonlines

# train_path = get_all_files_from_path("../data/COLIEE2024statute_data-English/train")
# out_trainpath = "../data/COLIEE2024statute_data-English/train.jsonl"

# test_path = get_all_files_from_path("../data/COLIEE2024statute_data-English/test")
# out_testpath = "../data/COLIEE2024statute_data-English/"
# def xml2json(files, out_path):
#     result = []
#     for file in files:
#         data = load_samples(file)
#         for k in data:
#             item = {}
#             if 'index' not in k:
#                 print(k)
#             item.update({"id": k['index']})
#             item.update({"content": zeroShot.replace("{{premise}}", k["result"]).replace("{{hypothesis}}", k["content"])})
#             if k["label"] == "Y": item.update({"label": "true"})
#             else: item.update({"label": "false"})
#             result.append(item)
#     if "R0" in files[0]:
#         out_path = out_path+file.split("/")[-1].replace(".xml", "")+".jsonl"
#     with jsonlines.open(out_path, 'w') as writer:
#         writer.write_all(result)

# # xml2json(train_path, out_trainpath)
# xml2json([test_path[0]], out_testpath)
# xml2json([test_path[1]], out_testpath)
# xml2json([test_path[2]], out_testpath)
# xml2json([test_path[3]], out_testpath)
# xml2json([test_path[4]], out_testpath)


In [4]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train":f"{output_data}/train_full.jsonl", 
                                          "riteval_R01_en": f"{output_data}/riteval_R01_en.jsonl",
                                          "riteval_R02_en": f"{output_data}/riteval_R02_en.jsonl",
                                          "riteval_R03_en": f"{output_data}/riteval_R03_en.jsonl",
                                          "riteval_R04_en": f"{output_data}/riteval_R04_en.jsonl",
                                          "riteval_H30_en": f"{output_data}/riteval_H30_en.jsonl",
                                          "riteval_R05_en": f"{output_data}/riteval_R05_en.jsonl"})

print(dataset)


/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 3663
    })
    riteval_R01_en: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 111
    })
    riteval_R02_en: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 81
    })
    riteval_R03_en: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 109
    })
    riteval_R04_en: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 101
    })
    riteval_H30_en: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 70
    })
    riteval_R05_en: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 109
    })
})


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-xxl"

# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/home/congnguyen/drive/.cache")


In [6]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# cache = "/home/congnguyen/drive/"

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "../output/finetuned/zeroshot_aug_r32_peft_results_prompt20_batch64_epochs30"

config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto", 
                                              cache_dir="/home/congnguyen/drive/.cache", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, cache_dir="/home/congnguyen/drive/.cache")

# Load the Lora model
# model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id, device_map="auto")
model.eval()
print("Peft model loaded")


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:16<00:00,  1.41s/it]


Peft model loaded


In [7]:
def write_test(data, output_file):
    f = open(output_file, "w", encoding="utf-8")
    for k in data:
        f.write(k.strip()+" "+data[k]+" "+"CAPTAIN\n")
        
def load_jsonl(file):
    with open(file, encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    return data
    
test_data = load_jsonl("../data/COLIEE2024statute_data-English/fewshot.json/riteval_R03_en.jsonl")
test_label = {}
for item in test_data:
    test_label.update({item["index"]: item["label"]})
# test_label

In [8]:
from datasets import load_dataset
from random import randrange
import os

# for i in range(len(test_dataset["content"])):
# if not os.path.exists(output_folder):
# 	os.makedirs(output_folder)

files = ["riteval_R03_en"]
acc = {}
for file in files:
    count = 0
    data = dataset[file]
    result = {}
    for item in tqdm(data):
        id = item["id"]
        label = item["label"]
        content = item["content"]
        input_ids = tokenizer(content, return_tensors="pt").input_ids.cuda()
        outputs = model.generate(input_ids=input_ids, max_new_tokens=10)
        output_text = tokenizer.decode(outputs[0]).replace(content, "").split("\n")[-1]
        print(output_text)
        output_text = format_output(output_text)
        answer = "false"
        if  "true" in output_text or "yes" in output_text:
            answer = "true"
        if answer == label:
            count += 1
        if answer == "true": answer = "Y"
        else: answer = "N"
        result.update({id: answer})
    key = file.split("_")[1]
    # outfile_path = f"{output_folder}/{key}.task4.CAPTAIN2_{file}.xml"
    # write_test(result, outfile_path)
    acc.update({file: count/len(data)})
# write_json(outfile_path.replace(".xml", ".json"),acc)
    

  1%|█▉                                                                                                                                                                                                              | 1/109 [00:01<01:54,  1.06s/it]

<pad> false</s>


  2%|███▊                                                                                                                                                                                                            | 2/109 [00:01<01:11,  1.49it/s]

<pad> true</s>


  3%|█████▋                                                                                                                                                                                                          | 3/109 [00:01<00:56,  1.88it/s]

...

<pad> false</s>


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 108/109 [00:42<00:00,  2.71it/s]

<pad> false</s>


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [00:42<00:00,  2.55it/s]

<pad> true</s>


In [9]:
acc

{'riteval_R03_en': 0.8256880733944955}

In [12]:
# dataset["test1"][0]
count = 0
for k in result:
    if result[k] == test_label[k]:
        count += 1
print(count)
count/len(result)

91


0.8348623853211009

# FewShot

In [8]:
from sentence_transformers import SentenceTransformer, util
query_encoder = SentenceTransformer('facebook-dpr-question_encoder-single-nq-base')
passage_encoder = SentenceTransformer('facebook-dpr-ctx_encoder-single-nq-base')

def dpr(testfile=None, path="../data/COLIEE2024statute_data-English/train"):
    datas = get_all_files_from_path(path)
    corpus = []
    content = []
    labels = []
    for data in datas:
        if testfile != None and testfile in data:
            continue
        data = load_samples(data)
        for item in data:
            # corpus.append(item["result"].replace("\n", " ").strip())
            corpus.append(item["result"].strip())
            content.append(item["content"].strip().replace(".", ""))
            labels.append(item["label"].strip())
    print(len(corpus))
    retrival_passage_embeddings = passage_encoder.encode(corpus)
    content_passage_embeddings = passage_encoder.encode(content)
    return corpus, content, labels, retrival_passage_embeddings, content_passage_embeddings

corpus, content, labels, retrival_passage_embeddings, content_passage_embeddings = dpr()

695


In [9]:
def prompting(premise, hypothesis, template=None):
    text = template.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis)
    return text

def writefile(data, filename):
    # Serializing json
    json_object = json.dumps(data, indent=1)
    # Writing to sample.json
    with open(filename, "w") as outfile:
        outfile.write(json_object)
        
def few_shot_prompting(indexes, corpus, content, labels, prompt_template):
    result = ""
    for i in indexes:
        if "true or false" in prompt_template.lower():
            answer = "True"
            if "N" == labels[i]:
                answer = "False"
        else:
            answer = "Yes"
            if "N" == labels[i]:
                answer = "No"
        prompt = prompt_template.replace("{{premise}}", corpus[i]).replace('{{hypothesis}}', content[i]).replace('{{answer}}', answer)
        result += prompt
    return result
    

In [10]:
fewshot = "Document: {{premise}}\nQuestion: {{hypothesis}}? True or False\n{{answer}}\n"
template = "Document: {{premise}}\nQuestion: {{hypothesis}}? True or False "

In [11]:
from datasets import load_dataset
from random import randrange

result = {}
# Load dataset from the hub and get a sample
input_data_path = "../data/COLIEE2024statute_data-English/test"

files = get_all_files_from_path(input_data_path)
print(files)
acc = {}
# for i in range(len(test_dataset["content"])):
for file in files:
    count = 0
    data = load_samples(file)
    for item in tqdm(data):
        id = item["index"]
        label = item["label"]
        hypothesis = item["content"]
        premise = item["result"]
        few_shot = ""
    #     # inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    #     # outputs = model.generate(inputs, max_new_tokens=10)
        # =======================
    #     #Important: You must use dot-product, not cosine_similarity
        query_embedding = query_encoder.encode(premise)
        scores = util.dot_score(query_embedding, retrival_passage_embeddings)
        indexes = torch.topk(scores, 3).indices[0]
        few_shot = few_shot_prompting(indexes, corpus, content, labels, fewshot)
        text = few_shot + prompting(premise, hypothesis, template)
    #     ############################
        input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
        # outputs = model.generate(input_ids=input_ids, max_new_tokens=10, do_sample=True, top_p=0.0000001)
        outputs = model.generate(input_ids=input_ids, max_new_tokens=10)
        output_text = tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1]
        print(output_text, " - ", label)
        output_text = format_output(output_text)
        
        ##########################
        # output = []
        # query = hypothesis.split(".")
        # if len(query) > 1:
        #     for i in range(0, len(query)-1):
        #         text = prompting(premise, query[i], template)
        #         print(text)
        #         input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
        #         outputs = model.generate(input_ids=input_ids, max_new_tokens=10)
        #         output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
        #         output.append(output_text)
        # else:
        #     text = text = prompting(premise, hypothesis, template)
        #     # text = text.replace("\n", " ")
        #     input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
        #     outputs = model.generate(input_ids=input_ids, max_new_tokens=10)
        #     output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
        #     output.append(output_text)
    
        # output_text = output[0]
        # for k in output:
        #     if output_text != k:
        #         output_text = "false"
        # print(output, output_text, label)
        #############################
        if count < 1:
            print(text)
            
        answer = "N"
        if  "true" in output_text or "yes" in output_text:
            answer = "Y"
        if answer == label:
            count += 1
        if answer == "true": answer = "Y"
        else: answer = "N"
        result.update({id: answer})
    acc.update({file.split("/")[-1]: count/len(data)})    


['../data/COLIEE2024statute_data-English/test/riteval_H30_en.xml', '../data/COLIEE2024statute_data-English/test/riteval_R04_en.xml', '../data/COLIEE2024statute_data-English/test/riteval_R05_en.xml', '../data/COLIEE2024statute_data-English/test/riteval_R01_en.xml', '../data/COLIEE2024statute_data-English/test/riteval_R02_en.xml', '../data/COLIEE2024statute_data-English/test/riteval_R03_en.xml']


  1%|██▏                                                                                                                                                         | 1/70 [00:00<00:49,  1.39it/s]

<pad> true</s>  -  Y
Document: Article 3
(1) The enjoyment of private rights commences at birth.
(2) Unless otherwise prohibited by applicable laws, regulations, or treaties, foreign nationals enjoy private rights.
Question: An unborn child may not be given a gift on the donor's death? True or False
True
Document: Article 176
The creation and transfer of a real right becomes effective solely by the manifestations of intention of the parties.
Article 177
Acquisitions of, losses of and changes in real rights on immovables may not be duly asserted against any third parties, unless the same are registered pursuant to the applicable provisions of the Real Property Registration Act (Act No. 123 of 2004) and other laws regarding registration.
Question: If a mortgage creation contract has the agreement of the mortgagee and of the mortgagor that has ownership of the subject matter of the mortgage, it will be effective even if it is not in writing and there is no registration of its creation? Tr

  3%|████▍                                                                                                                                                       | 2/70 [00:01<00:37,  1.82it/s]

<pad> false</s>  -  N


...

<pad> false</s>  -  N


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 108/109 [01:22<00:00,  1.42it/s]

<pad> true</s>  -  Y


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [01:23<00:00,  1.30it/s]

<pad> true</s>  -  N


In [12]:
acc

{'riteval_H30_en.xml': 0.9,
 'riteval_R04_en.xml': 0.801980198019802,
 'riteval_R05_en.xml': 0.44036697247706424,
 'riteval_R01_en.xml': 0.7297297297297297,
 'riteval_R02_en.xml': 0.7407407407407407,
 'riteval_R03_en.xml': 0.7889908256880734}